In [11]:
import pandas as pd
import numpy as np

# Grupos canónicos 

In [12]:
C2v = pd.DataFrame(
    data = {
        "E":            [1, 1, 1, 1, 1], 
        "C_2^z":       [1, 1, 1, -1, -1],
        "σ_v^{xz}" :   [1, 1, -1, 1, -1],
        "σ_v^{yz}":    [1, 1, -1, -1, 1],
        "Traslaciones": [" ","z", " ", "x", "y"],
        "Rotaciones": [" ",  " ", "Rz", "Rx", "Ry"],
        "Dobles": [" ", " x^2, y^2, z^2", "xy", "xz", "yz"]
        }, 
    index= ["g(c)","A1", "A2", "B1", "B2"])
C2v

,E,C_2^z,σ_v^{xz},σ_v^{yz},Traslaciones,Rotaciones,Dobles
g(c),1,1,1,1,,,
A1,1,1,1,1,z,,"x^2, y^2, z^2"
A2,1,1,-1,-1,,Rz,xy
B1,1,-1,1,-1,x,Rx,xz
B2,1,-1,-1,1,y,Ry,yz


In [13]:
C3v = pd.DataFrame(
    data = {
        "E":    [1, 1,1,2], 
        "2C_3": [2, 1,1,-1],
        "3σ_v" :[3, 1, -1, 0],
        "Traslaciones": [" ","z", " ", "(x, y)"],
        "Rotaciones": [" ", " ", "Rz", "(Rx, Ry)"],
        "Dobles": [" ", "z^2, x^2 + y^2", " ", "(xy, x^2 - y^2), (xz, yz)"]
        }, 
    index= ["g(c)","A1", "A2", "E"])
C3v

,E,2C_3,3σ_v,Traslaciones,Rotaciones,Dobles
g(c),1,2,3,,,
A1,1,1,1,z,,"z^2, x^2 + y^2"
A2,1,1,-1,,Rz,
E,2,-1,0,"(x, y)","(Rx, Ry)","(xy, x^2 - y^2), (xz, yz)"


In [14]:
C4v = pd.DataFrame(
    data = {
        "E":    [1, 1,1,1,1, 2], 
        "2C_4": [2, 1,1,-1,-1, 0],
        "C_2":  [1, 1,1,1,1,-2],
        "2σ_v": [2, 1,-1,1,-1,0],
        "2σ_d": [2, 1,-1,-1,1,0],
        "Traslaciones":    [" ", "z",              " ",  " ",       " ",   "(x, y)"],
        "Rotaciones":      [" ", " ",              "Rz", " ",       " ",   "(Rx, Ry)"],
        "Dobles":           [" ", "z^2, x^2 + y^2", " ",  "x^2-y^2", "xy ", "(xz, yz)"]
        }, 
    index= ["g(c)", "A1", "A2", "B1", "B2", "E"])

C4v


,E,2C_4,C_2,2σ_v,2σ_d,Traslaciones,Rotaciones,Dobles
g(c),1,2,1,2,2,,,
A1,1,1,1,1,1,z,,"z^2, x^2 + y^2"
A2,1,1,1,-1,-1,,Rz,
B1,1,-1,1,1,-1,,,x^2-y^2
B2,1,-1,1,-1,1,,,xy
E,2,0,-2,0,0,"(x, y)","(Rx, Ry)","(xz, yz)"


In [15]:
def ProductoDirecto(grupo, rep1, rep2):
    return  grupo.loc[rep1][:-3]* grupo.loc[rep2][:-2]

def EncontrarRepresentacion(grupo, repre):
    return [rep for rep in grupo.index if grupo.loc[rep][:-2].equals(repre)][0]

def SumaDirecta(group, rep1, rep2):
    return group.loc[rep1][:-2] + group.loc[rep2][:-2]

def RepresentacionTraslacional(grupo):
    #Obtenemos las irrep que contienen traslaciones
    traslaciones = grupo.Traslaciones[1:]
    #Seleccionamos las traslaciones, dado por su valor " " en el data frame
    irrep_tralacionales = traslaciones.iloc[np.where(traslaciones != " ")[0]].index
    #Se suman las primeras dos irreps
    suma = SumaDirecta(grupo, irrep_tralacionales[0], irrep_tralacionales[1])[:-1]
    #Se suma el resto
    for irr in irrep_tralacionales[2:]:
        suma = suma + C4v.loc[irr][:-3]

    return suma

def RepresentacionRotacional(grupo):
    #Obtenemos las irrep que contienen traslaciones
    rotaciones = grupo.Rotaciones[1:]
    #Seleccionamos las traslaciones, dado por su valor " " en el data frame
    irrep_rotacionales = rotaciones.iloc[np.where(rotaciones != " ")[0]].index
    #Se suman las primeras dos irreps
    suma = SumaDirecta(grupo, irrep_rotacionales[0], irrep_rotacionales[1])[:-1]
    #Se suma el resto
    for irr in irrep_rotacionales[2:]:
        suma = suma + C4v.loc[irr][:-3]

    return suma

def RepresentacionAND(grupo, caracteres_AND):
    rep_AND = grupo.iloc[1][:-3]  #Copiamos la representcion irreducible totalmente simétrica A1 del grupo
    
    for i in range(len(caracteres_AND)):
        rep_AND[rep_AND.index[i]] = caracteres_AND[i]
    rep_AND.name = "AND"
    return rep_AND

def RepresentacionTotal(grupo, caracteresAND):
    return RepresentacionTraslacional(grupo)* RepresentacionAND(grupo, caracteresAND)

def RepresentacionVibracional(grupo, caracteresAND):
    return RepresentacionTotal(grupo, caracteresAND) - RepresentacionTraslacional(grupo) - RepresentacionRotacional(grupo) 

def Reducir(grupo, representacion):
    h = np.sum(grupo.loc["g(c)"][:-3])
    irreps = grupo.index[1:]

    coeficientes = np.zeros(len(irreps))
    formato = " "
    i=0
    for irrep in irreps:
        coeficientes[i] = int(np.sum(grupo.loc["g(c)"][:-3]* grupo.loc[irrep][:-3]*representacion )/h)
        if coeficientes[i]!=0:
            formato = formato + str(coeficientes[i]) + irrep + " + "
        i+=1

    return coeficientes, formato

def RepresentacionesRaman(grupo, representacion):
    #Las Raman son activas en productos dobles
    dobles = grupo.Dobles[1:]
    # Seleccionamos las traslaciones, dado por su valor " " en el data frame
    irrep_dobles = dobles.iloc[np.where(dobles != " ")[0]].index

    irreducibles = Reducir(grupo, representacion)[1]
    raman=[]
    
    for rep in irrep_dobles:        
        if rep in irreducibles:
    
            raman.append(rep)     
            
    return raman

def RepresentacionesInfrarrojo(grupo, representacion):
    #En el infrarrojo se activan las representaciones que transforman acorde a traslaciones
    traslaciones = grupo.Traslaciones[1:]
    # Seleccionamos las traslaciones, dado por su valor " " en el data frame
    irrep_traslaciones = traslaciones.iloc[np.where(traslaciones != " ")[0]].index

    #Encontramos la forma irreducible de la representación para la que buscamos la activación en infrarrojo
    irreducibles = Reducir(grupo, representacion)[1]
    infra_rojo =[]
    
    for rep in irrep_traslaciones:        
        if rep in irreducibles:
    
            infra_rojo.append(rep)     
            
    return infra_rojo
    


Para una molécula con simetría $C_{4v}$,hay que hallar la representación reducible de los modos normales de vibración $\Gamma_{vib}$ y después reducirla. Para ello, hallamos las representaciones reducibles Traslacional y Rotacional

In [16]:
total = RepresentacionTotal()

TypeError: RepresentacionTotal() missing 2 required positional arguments: 'grupo' and 'caracteresAND'

In [ ]:
rep_traslacional = RepresentacionTraslacional(C4v)
rep_traslacional

E        3
2C_4     1
C_2     -1
2σ_v     1
2σ_d     1
dtype: object

In [ ]:
# Ingresar la representaciónde átomos no desplazados

caracteres_AND = [6, 2, 2, 4, 2]
RepresentacionTotal(C4v, caracteres_AND)


E       18
2C_4     2
C_2     -2
2σ_v     4
2σ_d     2
dtype: object

In [ ]:
RepresentacionVibracional(C4v, caracteres_AND)

E       12
2C_4     0
C_2      0
2σ_v     4
2σ_d     2
dtype: object

In [ ]:
round(np.sum(C4v.loc["g(c)"][:-3]* C4v.loc["A1"][:-3]*RepresentacionVibracional(C4v, caracteres_AND) )/8)

3

In [ ]:
rep_vibracional = RepresentacionVibracional(C4v, caracteres_AND)
rep_vibracional

E       12
2C_4     0
C_2      0
2σ_v     4
2σ_d     2
dtype: object

In [ ]:
Reducir(C4v, rep_vibracional)

(array([3., 0., 2., 1., 3.]), ' 3.0A1 + 0.0A2 + 2.0B1 + 1.0B2 + 3.0E + ')

In [ ]:
C4v.loc["A1"][:-3] 

E       1
2C_4    1
C_2     1
2σ_v    1
2σ_d    1
Name: A1, dtype: object

In [ ]:
caracteres_orpbitales_p = [12, 0, 0, 2, 0]

representacionPxyz = RepresentacionAND(C4v, caracteres_orpbitales_p)
Reducir(C4v, representacionPxyz)

(array([2., 1., 2., 1., 3.]), ' 2.0A1 + 1.0A2 + 2.0B1 + 1.0B2 + 3.0E + ')

In [ ]:


RepresentacionesRaman(C4v, rep_vibracional)

['A1', 'B1', 'B2', 'E']

In [ ]:


RepresentacionesInfrarrojo(C4v, rep_vibracional)

['A1', 'E']

In [ ]:
rep_vibracional

E       12
2C_4     0
C_2      0
2σ_v     4
2σ_d     2
dtype: object

### Molécula de agua

La molécula de agua pertenece al grupo de simetría $C_{2v}$

In [ ]:
C2v

,E,C_2^z,σ_v^{xz},σ_v^{yz},Traslaciones,Rotaciones,Dobles
g(c),1,1,1,1,,,
A1,1,1,1,1,z,,"x^2, y^2, z^2"
A2,1,1,-1,-1,,Rz,xy
B1,1,-1,1,-1,x,Rx,xz
B2,1,-1,-1,1,y,Ry,yz


In [17]:
rep_and = RepresentacionAND(C2v, [2, 0, 2, 0])
Reducir(C2v, rep_and)
   

(array([1., 0., 1., 0.]), ' 1.0A1 + 1.0B1 + ')

In [ ]:
"""
coeficientes = np.zeros(len(irreps))
formato = " "
i=0
for irrep in irreps:
    coeficientes[i] = int(np.sum(grupo.loc["g(c)"][:-3]* grupo.loc[irrep][:-3]*representacion )/h)
    if coeficientes[i]!=0:
        formato = formato + str(coeficientes[i]) + irrep + " + "
    i+=1
"""
Reducir(C2v, rep_and)
         

KeyError: 'E'

In [ ]:
E

NameError: name 'E' is not defined